In [5]:
import os
import pandas as pd
import numpy as np
import glob

In [23]:
pd.set_option('display.max_rows', None)

In [ ]:
dfList = []
fmask = '/home/mila/reit/*.xls*' 

for f in glob.glob(fmask): 
    df = pd.read_excel(f)
    comp = df.iloc[0,0]
    df.dropna(subset = ["Unnamed: 1"], inplace=True)
    df = df.reset_index(drop=True)
    df = df.drop([0]).reset_index(drop=True)
    df1 = df.loc[df['Unnamed: 0'].str.contains('   ')]
    df2 = df.loc[~df['Unnamed: 0'].str.contains('   ')]
    l = list(df2.index.values.tolist())
    l_mod = [0] + l + [max(l)+len(df)-1]
    list_of_dfs = [df.iloc[l_mod[n]:l_mod[n+1]] for n in range(len(l_mod)-1)]
    for li in np.arange(1,(len(l)+1)):
        list_of_dfs[li]['segment'] = list_of_dfs[li].iloc[0,0]
        list_of_dfs[li] = list_of_dfs[li].reset_index(drop=True)
        list_of_dfs[li] = list_of_dfs[li].drop([0])
    df3 = pd.concat(list_of_dfs).reset_index(drop=True)
    df3['MSA'] = df3['Unnamed: 0']
    #not USA
    df4 = df3[df3['MSA'].str.contains('USA')==False]
    if  (df4['MSA'].str.contains(',')).any():
        df4[['city','country']] = df4['MSA'].str.split(', ',expand=True)
        df4['state'] = 0
        df4.loc[df4.index[df4['country'].isnull()].tolist(),'state'] = df4['city']
        df4.loc[df4.index[df4['country'].isnull()].tolist(),'city'] = ' '
        df4.loc[df4.index[df4['country'].isnull()].tolist(),'country'] = df4['state']
        df4['state'] = ''
    elif (df4['MSA'].str.contains(',')==False).any():
        df4['country'] = df4['MSA']
        df4['state'] = ''
        df4['city'] = ''
    #USA
    if (df3['MSA'].str.contains('USA')).any():
        df5 = df3[df3['MSA'].str.contains('USA')]
        df5[['city','state','country']] = df5['MSA'].str.split(', ',expand=True)
        df5 = df5.replace(np.nan, '', regex=True)
        df5.loc[df5.index[df5['state'].str.contains('USA')].tolist(),'country']=df5['state']
        df5['state_add']=df5['state']
        df5.loc[df5.index[df5['state_add'].str.contains('USA')].tolist(),'state']=df5['city']
        df5.loc[df5.index[df5['state_add'].str.contains('USA')].tolist(),'city']=' '
        del df5['state_add']
        df5.loc[df5.index[df5['city'].str.contains('USA')].tolist(),'country']=df5['city']
        df5.loc[df5.index[df5['city'].str.contains('USA')].tolist(),'city']=' '
        df3 = df5.append(df4, ignore_index=True)
    elif (df3['MSA'].str.contains('USA')==False).any():
        df3 = df4
    
    df3['REITS'] = comp
    df3["REITS"] = df3["REITS"].str.split("|", expand=True)[0]
    df3['number'] = df3['Unnamed: 1']
    df3['owned_size_size'] = df3['Unnamed: 2']
    df3['owned_size_exposure'] = df3['Unnamed: 3']
    df3['owned_other_size_size'] = df3['Unnamed: 4']
    df3['owned_other_size_unit'] = df3['Unnamed: 5']
    df3['owned_other_size_exposure'] = df3['Unnamed: 6']
    df3 = df3[['REITS','segment','country','state','city','number','owned_size_size','owned_size_exposure','owned_other_size_size',
          'owned_other_size_unit','owned_other_size_exposure']]
    df3['file'] = f
    dfList.append(df3) 

appended_data = pd.concat(dfList)
appended_data = appended_data.replace(np.nan, '', regex=True)
appended_data["file"] = appended_data["file"].str.split("Asia", expand=True)[1]
appended_data['city'] = appended_data['city'].str[8:]
appended_data['country'] = appended_data['country'].replace(' ', '', regex=True)
appended_data.to_excel('reits_europe-asia.xlsx')

In [175]:
dfList = []
fmask = '/home/mila/reit/*.xls*' 
#newpath = 'C:\\Path\\To\\New\\Folder'

for f in glob.glob(fmask): 
    df = pd.read_excel(f)
    comp = df.iloc[0,0]
    df.dropna(subset = ["Unnamed: 1"], inplace=True)
    df = df.reset_index(drop=True)
    df = df.drop([0]).reset_index(drop=True)
    df1 = df.loc[df['Unnamed: 0'].str.contains('   ')]
    df2 = df.loc[~df['Unnamed: 0'].str.contains('   ')]
    l = list(df2.index.values.tolist())
    l_mod = [0] + l + [max(l)+len(df)-1]
    list_of_dfs = [df.iloc[l_mod[n]:l_mod[n+1]] for n in range(len(l_mod)-1)]
    for li in np.arange(1,(len(l)+1)):
        list_of_dfs[li]['segment'] = list_of_dfs[li].iloc[0,0]
        list_of_dfs[li] = list_of_dfs[li].reset_index(drop=True)
        list_of_dfs[li] = list_of_dfs[li].drop([0])
    df3 = pd.concat(list_of_dfs).reset_index(drop=True)
    df3['MSA'] = df3['Unnamed: 0']
    df3['country'] = 'USA'

    df3[['city','state']] = df3['MSA'].str.split(', ',expand=True)
    
    if (df3['state'].str.contains('USA')).any():
        df3['country'] = df3['state']
        df3['state'] = df3['city']
        df3['city'] = ''
     
    if (df3['country'].str.contains('USA')==False).any():
        df3['city'] = df3['state']
        df3['state'] = ''
        
    if (df3['state'].str.contains('Canada')).any():
        df3['country'] = df3['state']
        df3['state'] = ''
    

    df3['REITS'] = comp
    df3["REITS"] = df3["REITS"].str.split("|", expand=True)[0]
    df3['number'] = df3['Unnamed: 1']
    df3['owned_size_size'] = df3['Unnamed: 2']
    df3['owned_size_exposure'] = df3['Unnamed: 3']
    df3['owned_other_size_size'] = df3['Unnamed: 4']
    df3['owned_other_size_unit'] = df3['Unnamed: 5']
    df3['owned_other_size_exposure'] = df3['Unnamed: 6']
    df3 = df3[['REITS','segment','country','state','city','number','owned_size_size','owned_size_exposure','owned_other_size_size',
          'owned_other_size_unit','owned_other_size_exposure']]
    df3['file'] = f
    dfList.append(df3)
    
appended_data = pd.concat(dfList).reset_index(drop=True)
appended_data['state'] = appended_data['state'].replace(np.nan, 'rho', regex=True)
appended_data.loc[appended_data.index[appended_data['state'].str.contains('rho')].tolist(),'country']=appended_data['city']
appended_data.loc[appended_data.index[appended_data['state'].str.contains('rho')].tolist(),'city']=''
appended_data.loc[appended_data.index[appended_data['state'].str.contains('rho')].tolist(),'state']=''
appended_data["file"] = appended_data["file"].str.split("reit", expand=True)[1]
appended_data['city'] = appended_data['city'].str[8:]
appended_data['country'] = appended_data['country'].replace(' ', '', regex=True)
appended_data['state'] = appended_data['state'].replace(' ', '', regex=True)
appended_data.to_excel('reit.xlsx')

/home/mila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dfList = []
fmask = '/home/mila/reit/Europe_Asia/spec/states/*.xls*' 
#newpath = 'C:\\Path\\To\\New\\Folder'

for f in glob.glob(fmask): 
    df = pd.read_excel(f)
    comp = df.iloc[0,0]
    df.dropna(subset = ["Unnamed: 1"], inplace=True)
    df = df.reset_index(drop=True)
    df = df.drop([0]).reset_index(drop=True)
    df1 = df.loc[df['Unnamed: 0'].str.contains('   ')]
    df2 = df.loc[~df['Unnamed: 0'].str.contains('   ')]
    l = list(df2.index.values.tolist())
    l_mod = [0] + l + [max(l)+len(df)-1]
    list_of_dfs = [df.iloc[l_mod[n]:l_mod[n+1]] for n in range(len(l_mod)-1)]
    for li in np.arange(1,(len(l)+1)):
        list_of_dfs[li]['segment'] = list_of_dfs[li].iloc[0,0]
        list_of_dfs[li] = list_of_dfs[li].reset_index(drop=True)
        list_of_dfs[li] = list_of_dfs[li].drop([0])
    df3 = pd.concat(list_of_dfs).reset_index(drop=True)
    df3['MSA'] = df3['Unnamed: 0']
    #not USA
    df4 = df3[df3['MSA'].str.contains('USA')==False]
    if  (df4['MSA'].str.contains(',')).any():
        df4[['city','country']] = df4['MSA'].str.split(', ',expand=True)
        df4['state'] = 0
        df4.loc[df4.index[df4['country'].isnull()].tolist(),'state'] = df4['city']
        df4.loc[df4.index[df4['country'].isnull()].tolist(),'city'] = ' '
        df4.loc[df4.index[df4['country'].isnull()].tolist(),'country'] = df4['state']
        df4['state'] = ''
    elif (df4['MSA'].str.contains(',')==False).any():
        df4['country'] = df4['MSA']
        df4['state'] = ''
        df4['city'] = ''
    #USA
    if (df3['MSA'].str.contains('USA')).any():
        df5 = df3[df3['MSA'].str.contains('USA')]
        df5[['city','state','country']] = df5['MSA'].str.split(', ',expand=True)
        df5 = df5.replace(np.nan, '', regex=True)
        df5.loc[df5.index[df5['state'].str.contains('USA')].tolist(),'country']=df5['state']
        df5['state_add']=df5['state']
        df5.loc[df5.index[df5['state_add'].str.contains('USA')].tolist(),'state']=df5['city']
        df5.loc[df5.index[df5['state_add'].str.contains('USA')].tolist(),'city']=' '
        del df5['state_add']
        df5.loc[df5.index[df5['city'].str.contains('USA')].tolist(),'country']=df5['city']
        df5.loc[df5.index[df5['city'].str.contains('USA')].tolist(),'city']=' '
        df3 = df5.append(df4, ignore_index=True)
    elif (df3['MSA'].str.contains('USA')==False).any():
        df3 = df4
    
    df3['REITS'] = comp
    #df3["REITS"] = df3["REITS"].str.split("|", expand=True)[0]
    df3['number'] = df3['Unnamed: 1']
    df3['owned_size_size'] = df3['Unnamed: 2']
    df3['owned_size_exposure'] = df3['Unnamed: 3']
    df3['owned_other_size_size'] = df3['Unnamed: 4']
    df3['owned_other_size_unit'] = df3['Unnamed: 5']
    df3['owned_other_size_exposure'] = df3['Unnamed: 6']
    df3 = df3[['REITS','segment','country','state','city','number','owned_size_size','owned_size_exposure','owned_other_size_size',
          'owned_other_size_unit','owned_other_size_exposure']]
    df3['file'] = f
    dfList.append(df3) 

appended_data = pd.concat(dfList)
appended_data = appended_data.replace(np.nan, '', regex=True)
appended_data["file"] = appended_data["file"].str.split("Asia", expand=True)[1]
appended_data['city'] = appended_data['city'].str[8:]
appended_data['country'] = appended_data['country'].replace(' ', '', regex=True)
appended_data.to_excel('reits_europe-asia1.xlsx')